In [1]:
import numpy as np
import numba

In short the problem is: 

Forgetful guy goes to home and office and back, repeatedly.  He has one umbrella at home (or more generally x umbrellas at home) and one umbrella at office (or more generally y umbrellas at office).  

Each time it rains, (and it does, with some probability p at the time of each trip) he takes umbrella -- if there is one-- from current location to next location and leaves it there.  I.e. he only moves an umbrella from one location to the next if it is raining during that trip.  However, if there are no umbrellas there during that trip, he gets soaked.  On average how many trips does he make before getting soaked?
- - - 
The problem then asks us to consider this over many different probability of raining values in $(0, 1)$.  

This notebook first solves this using a simulation -- as is the preferred method for "Digital Dice".  It then solves for a more exact solution using an absorbing state markov chain -- first in a simple but wasteful setup, then at the end in a simplified and more flexible setup.  Reference "markov_chains_absorbing_state_recut.ipynb" for more info on absorbing state markov chains.

One open item -- it feels like there may be a way to extract an analytic (scalar) solution from the simplified chain, for time until absorbtion.  However, such a solution is currently not coming to mind.

In [2]:
@numba.jit(nopython= True)
def run_sim(p, starting_home_umbrellas = 1, starting_office_unmbrellas = 1, n_trials= 100000):
    total_moves_counter = 0
    umbrella_array = np.zeros(2) 
    
    for _ in range(n_trials):
        umbrella_array[0] = starting_home_umbrellas
        umbrella_array[1] = starting_office_unmbrellas
        local_moves_counter = 0
        curstate = 0 # at home to start
        next_state = 1
        while True:
            local_moves_counter += 1
            if np.random.random() <= p:
                # i.e. it rains
                if umbrella_array[curstate] == 0:
                    break
                else:
                    umbrella_array[curstate] -= 1
                    umbrella_array[next_state] += 1
                
            curstate, next_state = next_state, curstate # swap after a move
        total_moves_counter += local_moves_counter
    return total_moves_counter / n_trials

    


# Below is explicit markov chain formulation, for the 1 umbrella at home, 1 umbrella at office case




In [3]:
# The two functions in this cell come from "markov_chains_absorbing_state_recut.ipynb"

def partion_transition_matrix(A, first_absorbing_col=-2, last_absorbing_col = -1):
    """
    this takes a transition matrix and slices it into two slices.  see dartmoth_markov_Lecture14.pdf page 11.
    Also note that the slides use a tranposed A... (see how their rows sum to one, not the cols)
    
    Note, for now I just have this setup to accomodate absorbing states in EITHER the beginning of the matrix or the end,
    but NOT both
    
    note this is back to using a zero indexing convention
    
    A must be square
    """    
    assert A.shape[0] == A.shape[1]
    m = A.shape[0]
    
    if first_absorbing_col < 0:
        first_absorbing_col += m
    if last_absorbing_col < 0:
        last_absorbing_col += m
    newA = A.T 
    assert first_absorbing_col <= last_absorbing_col, "check that first_absorbing_col and last_absorbing_col are sensible"
    if first_absorbing_col == 0:
        raise NotImplementedError
    else:
        N_inv = np.identity(first_absorbing_col) - np.zeros((first_absorbing_col, first_absorbing_col), dtype= np.float64)
        R = np.zeros((first_absorbing_col - 0, last_absorbing_col - first_absorbing_col + 1), dtype= np.float64)
        for i in range(first_absorbing_col):
            for j in range(first_absorbing_col):
                N_inv[i,j] -= newA[i,j]
        for i in range(first_absorbing_col): 
            for j in range(last_absorbing_col - first_absorbing_col + 1):
                R[i, j] += newA[i, first_absorbing_col + j]
    return N_inv, R


def get_variance_alternative(N_inverse, mean_vector):
    """
    This returns a variance vector for a start at one of the non-absorbing states... and for starting 100% of the time 
    only at that state, no where else....
    
    This seems much more opaque... 
    but it is computationally better has the benefit of not needing to explicitly invert any matrices
    
    See writeup for derivation
    """    
    right_side = 2 * mean_vector - N_inverse @ (mean_vector + mean_vector**2) 
    a = np.linalg.solve(N_inverse, 2 * mean_vector)
    b = - (mean_vector + mean_vector**2)
    x = a + b
    return x
  


In [4]:
@numba.jit(nopython= True)
def matrix_setter_helper(matrixA, state_from, state_to, probability):
    # in form of Ax = b
    matrixA[state_to, state_from] = probability

# sketch for below "create_matrix" function

![basic markov chain](illustrations/digital_dice10_raw_model_sketch.jpeg)

In [5]:

@numba.jit(nopython= True)
def create_matrix(p):
    """
    This creates a simple, (but somewhat rigid and wasteful) transition matrix to solve the problem
    See create_simplified_A for a simpler and more powerful approach
    
    See above picture that this maps to
    """
    
    A = np.zeros((7,7))
    q = 1 - p

    matrix_setter_helper(A, 0, 1, q)
    matrix_setter_helper(A, 1, 0, q)
    matrix_setter_helper(A, 1, 2, p)
    matrix_setter_helper(A, 2, 1, p)
    matrix_setter_helper(A, 2, 3, q)
    matrix_setter_helper(A, 3, 2, q)
    matrix_setter_helper(A, 3, -1, p)
    matrix_setter_helper(A, 0, 4, p)
    matrix_setter_helper(A, 4, 0, p)
    matrix_setter_helper(A, 4, 5, q)
    matrix_setter_helper(A, 5, 4, q)
    matrix_setter_helper(A, 5, -1, p)
    matrix_setter_helper(A, -1, -1, 1)
    
    return A

A = create_matrix(0.2)
N_inv, R = partion_transition_matrix(A, first_absorbing_col=-1, last_absorbing_col=-1) 

print(N_inv, "\n")

print(N_inv.shape, "the shape")
ones_v = np.ones(N_inv.shape[0])

print("answer of E[T]")
the_mean_vector =  np.linalg.solve(N_inv, ones_v)
print(the_mean_vector)

print("\n", R, "That's R","\n")
print("for absorbtion destination, the solution is")
print(np.linalg.solve(N_inv, R))
    
print("technically variance in time until absorbtion was not asked in the problem, but it is shown below for reference")
print("\n", "variance in time until absorbtion is shown below")
print(get_variance_alternative(N_inv, the_mean_vector))
print( " ")


[[ 1.  -0.8  0.   0.  -0.2  0. ]
 [-0.8  1.  -0.2  0.   0.   0. ]
 [ 0.  -0.2  1.  -0.8  0.   0. ]
 [ 0.   0.  -0.8  1.   0.   0. ]
 [-0.2  0.   0.   0.   1.  -0.8]
 [ 0.   0.   0.   0.  -0.8  1. ]] 

(6, 6) the shape
answer of E[T]
[ 22.5  22.5  17.5  15.   17.5  15. ]

 [[ 0. ]
 [ 0. ]
 [ 0. ]
 [ 0.2]
 [ 0. ]
 [ 0.2]] That's R 

for absorbtion destination, the solution is
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]
technically variance in time until absorbtion was not asked in the problem, but it is shown below for reference

 variance in time until absorbtion is shown below
[ 346.25  346.25  326.25  310.    326.25  310.  ]
 


In [6]:
# full chain only implemented for case of 1 umbrella at home and 1 umbrella at office
# this runs the simulation and numerically exact answer, below, as suggested in the original problem

p_vec = np.linspace(0.01, 1, endpoint=False, num=99 )

for p in p_vec:
    sim_result = run_sim(p)
    A = create_matrix(p)
    N_inv, R = partion_transition_matrix(A, first_absorbing_col=-1, last_absorbing_col=-1) 
    ones_v = np.ones(N_inv.shape[0]) # a touch wasteful here
    the_mean_vector =  np.linalg.solve(N_inv, ones_v)
    
    print(sim_result, the_mean_vector[0], "probability = ",p )
    
    assert(abs(the_mean_vector[0] - the_mean_vector[1]) <= 0.0001) 
    # this assert can be ignored
    # it has been put in place for symmetry checking



400.88783 402.02020202 probability =  0.01
202.41981 202.040816327 probability =  0.02
135.78751 135.395189003 probability =  0.03
101.9265 102.083333333 probability =  0.04
82.06599 82.1052631579 probability =  0.05
68.99457 68.7943262411 probability =  0.06
59.44357 59.2933947773 probability =  0.07
52.15435 52.1739130435 probability =  0.08
46.75081 46.6422466422 probability =  0.09
42.3543 42.2222222222 probability =  0.1
38.51979 38.6108273749 probability =  0.11
35.59199 35.6060606061 probability =  0.12
33.05008 33.0680813439 probability =  0.13
30.91296 30.8970099668 probability =  0.14
29.12171 29.0196078431 probability =  0.15
27.57649 27.380952381 probability =  0.16
25.92294 25.9390503189 probability =  0.17
24.57516 24.6612466125 probability =  0.18
23.52351 23.5217673814 probability =  0.19
22.52134 22.5 probability =  0.2
21.58735 21.5792646172 probability =  0.21
20.73535 20.7459207459 probability =  0.22
19.9243 19.9887069452 probability =  0.23
19.26414 19.298245614 p

# sketch for below "create_matrix" function

![better markov chain](illustrations/digital_dice10_simple_model_sketch.jpeg)

In [7]:
@numba.jit(nopython= True)
def create_simplified_A(p, starting_umbrellas_at_home=1, starting_umbrellas_at_office=1):
    """
    This setup takes advantage of symmetry to create a smaller, simplified transition matrix
    See above sketch for the 1 umbrella at home, 1 at the office case
    
    note that for simplicity starting_umbrellas_at_home at must == starting_umbrellas_at_office
    However, the model could easily be tweaked to accomodate something else (so long as total number of umbrellas is even)
    
    E.g. if there were 3 umbrellas at home and 5 at the office, we'd model this as (4,4), and just get the associated
    mean time until absorbtion from a different starting state
    
    However, if the total number of umbrellas is odd, the same ideas apply, but some structual differences with the 
    indexing scheme, below, would be required.  This would entail complicating this function considerably  
    without adding much insight.  Hence this has been omitted.  
    
    Note: The underlying matrices are extremely sparse, which would suggest a sparse matrix makes sense for larger matrices.
    This is certainly correct.  However scipy.sparse leaves a lot to be desired.  If I wanted to further optimize this,
    I would do a sparse implementation in Julia.
    
    An open question is whether or not a technique like telescoping can be employed 
    to extract an analytic (scalar) solution to his problem
    
    Or perhaps bunching then telescoping to get an analytic bound / approximation
    
    """
    # assert(type(starting_umbrellas_at_home) == int)
    # starting_umbrellas_at_home should be an integer, 
    # though numba does not like it when I use an explict assert isinstance or assert type()
    assert(starting_umbrellas_at_home == starting_umbrellas_at_office)
    assert( 0 < p < 1)

    # for simplicity we start with the case where the number of umbrellas at home and office are the same
    # note that this setup easily lends
    # always a starting state, and absorbing state, then 2 * number of umbrellas at home
    q = 1 - p
    
    total_states = 1 + 1 + 2*starting_umbrellas_at_home
    A = np.zeros((total_states, total_states))
    
    matrix_setter_helper(A, 0, 0, q) # starting state    
    
    for idx in range(1, total_states):        
        if idx % 2 == 1: 
            # i.e. an odd idx
            prob_to_use = p
        else: 
            # i.e. and even idx
            prob_to_use = q
        
        matrix_setter_helper(A, idx, idx - 1, prob_to_use)
        matrix_setter_helper(A, idx - 1, idx, prob_to_use)
    
    ## cleanup for the absorbing state
    A[-2,-1] = 0
    A[-1,-1] = 1
    return A

In [8]:
R_umbrellas_at_each_location = 4
P_rain = 0.55

In [9]:
A = create_simplified_A(P_rain, R_umbrellas_at_each_location, R_umbrellas_at_each_location)
A

array([[ 0.45,  0.55,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.55,  0.  ,  0.45,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.45,  0.  ,  0.55,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.55,  0.  ,  0.45,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.45,  0.  ,  0.55,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.55,  0.  ,  0.45,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.45,  0.  ,  0.55,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.55,  0.  ,  0.45,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.45,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.55,  1.  ]])

In [10]:
N_inv, R = partion_transition_matrix(A, first_absorbing_col=-1, last_absorbing_col=-1) 
ones_v = np.ones(N_inv.shape[0]) # a touch wasteful here
the_mean_vector =  np.linalg.solve(N_inv, ones_v)
the_mean_vector[0]

89.898989898989882

In [11]:
N_inv

array([[ 0.55, -0.55,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.55,  1.  , -0.45,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.45,  1.  , -0.55,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  , -0.55,  1.  , -0.45,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -0.45,  1.  , -0.55,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  , -0.55,  1.  , -0.45,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.45,  1.  , -0.55,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.55,  1.  , -0.45],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.45,  1.  ]])

In [12]:
run_sim(P_rain, R_umbrellas_at_each_location, R_umbrellas_at_each_location, n_trials= 500000)

89.810326